In [1]:
import numpy as np
import torch
import string
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.utils.rnn import pad_packed_sequence as unpack
from torch.nn.utils.rnn import pack_padded_sequence as pack
import math
import torch.utils.data as data
import json
import os
import pandas as pd
import random
import copy
import torch.utils.data.sampler as sampler
import torch.optim.lr_scheduler as lr_scheduler
import pickle


In [5]:
        with open('./data/chat2.pickle',"rb") as f1:  
            chat_result=pickle.load(f1)
        
        with open('./data/audio_raw_feature_13.pickle',"rb") as f2:  
            audio=pickle.load(f2)
        with open('./data/video_raw_feature_sum_moving_average.pickle',"rb") as f2:  
            video=pickle.load(f2)
        with open('./label/label.pickle',"rb") as f4:  
            real_result=pickle.load(f4)

In [3]:
sample = ['102844412722519367','102844212429550795','102844401151219358','102844401154430631','102844412717014335','102844401153971877','102844224148503678','102844412722847048','102844401152857762','102844412707380528','102844212431516886','102844283027925085','102844412716227901','102844412710001974','102844294670878922','102844294670551241','102844283023599703','102844412704496937','102844235751783874','102844401152071328','102844412709674293','102844401153447587','102844224148896895','102844235746868664','102979081290790284','102844283027531868','102844212431975640','102844401155937960','102844212429092040','102844341906649746','102844412706987311','102844412721339716','102844212430402768','102844341905011343','102844235753356742','102844235750997440','102844412709346612','102844412705217835','102844235752963525','102844412712164667','102844412705545516','102844341912220311','102844341907370644','102844235749424575','102844212429419722','102844294669568199','102844212431779031','102844294666422466','102844224146472059','102844212428895431','102844212429747404','102844235748703677','102844224146930812','102844212430730450','102844294674876621','102844341909598870','102844283020453971','102844294670026952','102844412723174729','102844341904683662','102844283025696858','102844235747261881','102844401154168486','102844235748310460','102844412711836986','102844412723567946','102844235749031358','102844294674286796','102844294666881219','102844412716686654','102844294671796427','102844224145685626','102844412717407552','102844235751390657','102844401156069033','102904869420860038','102910307641576395','102844341905404560','102844341906977427','102844212430075086','102844412711116088','102844401153578660','102844294667405508','102844412706659630','102844212431058132','102844341902586509','102844401152267937','102844212430927059','102844412708953395','102844212429944013','102844341912679064','102844235753749959','102844341908026005','102844283023206486','102844224147717245','102844412704890154','102844212430599377','102844412711443769','102844235747982779']


In [7]:
for i in sample:
    print('==')
    print(i)
    print(len(chat_result[i]))
    print(len(audio[i]))
    print(len(video[i]))
    print(len(real_result[i]))

==
102844412722519367
1828
2069
2072
2069
==
102844212429550795
1799
1472
1475
1472
==
102844401151219358
2027
1934
1938
1934
==
102844401154430631
2089
1652
1656
1652
==
102844412717014335
1822
1906
1909
1906
==
102844401153971877
1895
1777
1779
1777
==
102844224148503678
2822
2074
2077
2074
==
102844412722847048
2522
2158
2158
2158
==
102844401152857762
2403
1443
1446
1443
==
102844412707380528
2160
1611
1617
1611
==
102844212431516886
1363
1818
1821
1818
==
102844283027925085
1987
2267
2270
2267
==
102844412716227901
2531
1745
1749
1745
==
102844412710001974
1801
1818
1822
1818
==
102844294670878922
1542
2196
2199
2196
==
102844294670551241
1563
2343
2346
2343
==
102844283023599703
2219
1473
1477
1473
==
102844412704496937
1719
2630
2633
2630
==
102844235751783874
2923
2416
2420
2416
==
102844401152071328
1549
1860
1864
1860
==
102844412709674293
1617
1860
1863
1860
==
102844401153447587
1822
1767
1770
1767
==
102844224148896895
1703
1805
1808
1805
==
102844235746868664
2019
1982
19

In [39]:
import torch.utils.data as data

class Mul_data(data.Dataset):
    def __init__(self,d_type):
        self.d_type=d_type

        with open('./data/chat_raw.pkl',"rb") as f1:  
            self.chat_result=pickle.load(f1)
        
        with open('./data/audio_raw_feature_13.pickle',"rb") as f2:  
            self.audio=pickle.load(f2)
        with open('./data/video_raw_feature_sum_moving_average.pickle',"rb") as f2:  
            self.video=pickle.load(f2)
        with open('./label/label.pickle',"rb") as f4:  
            self.real_result=pickle.load(f4)
            
        if d_type=='train':
            self.sample = ['102844412722519367','102844212429550795','102844401151219358','102844401154430631','102844412717014335','102844401153971877','102844224148503678','102844412722847048','102844401152857762','102844412707380528','102844212431516886','102844283027925085','102844412716227901','102844412710001974','102844294670878922','102844294670551241','102844283023599703','102844412704496937','102844235751783874','102844401152071328','102844412709674293','102844401153447587','102844224148896895','102844235746868664','102979081290790284','102844283027531868','102844212431975640','102844401155937960','102844212429092040','102844341906649746','102844412706987311','102844412721339716','102844212430402768','102844341905011343','102844235753356742','102844235750997440','102844412709346612','102844412705217835','102844235752963525','102844412712164667','102844412705545516','102844341912220311','102844341907370644','102844235749424575','102844212429419722','102844294669568199','102844212431779031','102844294666422466','102844224146472059','102844212428895431','102844212429747404','102844235748703677','102844224146930812','102844212430730450','102844294674876621','102844341909598870','102844283020453971','102844294670026952','102844412723174729','102844341904683662','102844283025696858','102844235747261881','102844401154168486','102844235748310460','102844412711836986','102844412723567946','102844235749031358','102844294674286796','102844294666881219','102844412716686654']
        if d_type=='val':
            self.sample = ['102844294671796427','102844224145685626','102844412717407552','102844235751390657','102844401156069033','102904869420860038','102910307641576395','102844341905404560','102844341906977427','102844212430075086','102844412711116088','102844401153578660','102844294667405508','102844412706659630']
        if d_type=='test':
            self.sample = ['102844212431058132','102844341902586509','102844401152267937','102844212430927059','102844412708953395','102844212429944013','102844341912679064','102844235753749959','102844341908026005','102844283023206486','102844224147717245','102844412704890154','102844212430599377','102844412711443769','102844235747982779']
            
        self.WeightedSampling=[]
        for i in self.sample:
            self.WeightedSampling.extend(copy.copy(self.real_result[str(i)]))
        
        sampling = np.array(self.WeightedSampling)
        neg_idx = np.where(sampling == 0)[0] #general
        pos_idx = np.where(sampling == 1)[0] #highlight
        sampling = sampling.astype(np.float32)
        
        sampling.fill(0)
        sampling[neg_idx] = len(sampling) / float(len(neg_idx))
       # self.WeightedSampling[pos_idx] = len(self.WeightedSampling) / float(len(pos_idx))
        sampling[pos_idx] = len(sampling) / float(len(pos_idx))
        self.WeightedSampling = sampling

        
        self.sum=np.insert(np.cumsum([len(self.audio[str(i)]) for i in self.sample]),0,0)
        print("data load fin")

        
    def __len__(self):
        return self.sum[-1]
    def __getitem__(self,index):
            vid=np.histogram(index,self.sum)#sum으로 누적으로 히스토그램이 깔려있음/ 그중에 index의 위치
            vid = np.where(vid[0]>0)[0][0]#몇번째 game을 쓸지!
            vframe=index-self.sum[vid]#그 게임 안에서의 몇번째 프레임인지
            game_id=str(self.sample[vid])

            window=[]#batch*7(window size)*3(highlight result)
            for idx in range(23): #7 : window size
                s_window=[]
                if vframe+idx<len(self.audio[game_id]):
                    s_window+=list((self.chat_result[game_id][vframe+idx]))#vframe의 chat
                    s_window+=list(self.audio[game_id][vframe+idx])#vframe의 image
                    s_window+=[self.video[game_id][vframe+idx]]
                else:
                    #s_window=[0,0,0]#padding value
                    s_window=[0]*1014
                window.append(s_window)


            label=int(self.real_result[game_id][vframe])
            return game_id,np.array(window),label

In [40]:
train=Mul_data('train')
val=Mul_data('val')

print(train[100])
sampler1 = torch.utils.data.sampler.WeightedRandomSampler(weights=train.WeightedSampling.tolist(), num_samples=44000)
train_loader=torch.utils.data.DataLoader(train,batch_size=32,sampler=sampler1)
# train_loader=torch.utils.data.DataLoader(train,batch_size=32)
val_loader=torch.utils.data.DataLoader(val,batch_size=128)

data load fin
data load fin
('102844412722519367', array([[ 0.        ,  0.        ,  0.        , ..., -2.99407219,
        -3.54347805,  0.01117067],
       [ 0.        ,  0.        ,  0.        , ..., -2.54867552,
        -3.03780565,  0.02793789],
       [ 0.        ,  0.        ,  0.        , ..., -0.24941073,
        -0.11264702,  0.03156882],
       ...,
       [ 0.        ,  0.        ,  0.        , ..., -2.46387674,
        -2.8452097 ,  0.02793295],
       [ 0.        ,  0.        ,  0.        , ..., -3.52012466,
        -4.43745678,  0.03501002],
       [ 0.        ,  0.        ,  0.        , ..., -5.56696138,
        -3.35553581,  0.04068482]]), 0)


In [41]:
train[100][1].shape

(23, 1014)

In [42]:
input_size=1014
hidden_size=128
length=7
num_layers=3
class LSTM(nn.Module):
    def __init__(self):
        
        super().__init__()
        self._clf1 = nn.LSTM(input_size, hidden_size,3,batch_first=True)
        self._lin = nn.Sequential(nn.Linear(hidden_size, hidden_size),
                                 nn.Linear(hidden_size,2))

    def forward(self, x):
        x=x.cuda()
        hidden = Variable(torch.zeros(num_layers,x.size(0),hidden_size)).cuda() # (num_layers * num_directions, batch, hidden_size)
        cell = Variable(torch.zeros(num_layers,x.size(0),hidden_size)).cuda() # (num_layers * num_directions, batch, hidden_size)        out,hidden = self._clf1(x,h0)
        out,hidden = self._clf1(x,(hidden,cell))#batch*7*3
        feature = out[:,-1,:]
        out = self._lin(out[:,-1,:])
        return out,feature

In [43]:
model=LSTM().cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), 0.01,momentum=0.9,weight_decay=1e-4)


In [44]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [45]:
def fmeasure(output, target):
    _, pred = output.topk(1, 1, True, True)
    pred = pred.view(-1,1)
    target = target.view(-1,1)

    #overlap = ((pred== 1) + (target == 1)).gt(1)
    #overlap = overlap.view(-1,1)
    TP = len(np.where((pred==1)&(target==1)==True)[0]) # True positive
    FP = len(np.where((pred==1)&(target==0)==True)[0]) # Condition positive = TP + FN
    TN = len(np.where((pred==0)&(target==0)==True)[0])
    FN = len(np.where((pred==0)&(target==1)==True)[0])

    
    #overlap_len = overlap.data.long().sum()
    pred_len = pred.data.long().sum()
    gt_len   =  target.data.long().sum()

    return TP,FP,TN,FN,pred_len, gt_len,pred

In [46]:
weight_dir='./C_data+A_data+V_data/'


In [ ]:
if not os.path.exists(weight_dir):
    os.makedirs(weight_dir)
with open(weight_dir+'train_result','a') as f:
    f.write('=====result=======\n')
f1_best=0
for epoch in range(100):
    losses = AverageMeter()
    top1 = AverageMeter()
    print(epoch)
    model.train()
    for i, (g,inputs,labels) in enumerate(train_loader):
        inputs=inputs.float()
        inputs=inputs.cuda()
        labels=labels.cuda()
        optimizer.zero_grad()
        out,_=model(inputs)
        out=out.cuda()
        loss=criterion(out,labels)
        loss.backward()
        optimizer.step()
        
    model.eval()
    val_losses=AverageMeter()
    acc=0
    gt_sum=0
    tp_sum=0
    fp_sum=0
    fn_sum=0
    acc=0
    sum=0
    pred_sum=0
    with open(weight_dir+'train_result','a') as f:

        with torch.no_grad():
            for it, (g,inputs,labels) in enumerate(val_loader):
                inputs=inputs.float()
                inputs=inputs.cuda()
                labels=labels.cuda()
                out,_=model(inputs)
                out=out.cuda()
                loss=criterion(out,labels)
                val_losses.update(loss,labels.size(0))
                TP,FP,TN,FN,pred_len, gt_len,pred=fmeasure(out.cpu(),labels.cpu())
                tp_sum += TP
                fp_sum += FP
                fn_sum += FN
                pred_sum += pred_len
                gt_sum += gt_len
                acc=acc+TP+TN
                sum+=len(out)
            if tp_sum>0 and fp_sum>0 and fn_sum>0:
                precision = tp_sum/(tp_sum+fp_sum)
                recall = tp_sum / (tp_sum+fn_sum)
                f1 = (2*precision*recall / (precision + recall))
                accuracy=acc/sum
                print("epoch {} train_loss : {} , val_loss : {},p {}, r {}, f {}\n".format(epoch,losses.avg,val_losses.avg,precision,recall,f1))
                f.write("epoch {} train_loss : {} , val_loss : {},p {}, r {}, f {}\n".format(epoch,losses.avg,val_losses.avg,precision,recall,f1))
                torch.save(model.state_dict(),'{}'.format(weight_dir+str(epoch)+"train"))
                if f1_best<f1:
                    f.write("== best epoch {} train_loss : {} , val_loss : {},p {}, r {}, f {}\n".format(epoch,losses.avg,val_losses.avg,precision,recall,f1))
                    torch.save(model.state_dict(),'{}'.format(weight_dir+"best"))
                    f1_best=f1

            else:
                print("epoch {} train_loss : {} , val_loss : {},p {}, r {}, f {}\n".format(epoch,losses.avg,val_losses.avg,0,0,0))
                f.write("epoch {} train_loss : {} , val_loss : {},p {}, r {}, f {}\n".format(epoch,losses.avg,val_losses.avg,0,0,0))
                torch.save(model.state_dict(),'{}'.format(weight_dir+str(epoch)+"train"))                
            







0


In [11]:
test=Mul_data('test')
test_loader=torch.utils.data.DataLoader(test,batch_size=32)
dataset=weight_dir+'best'
checkpoint=torch.load(dataset,map_location='cuda:0')
model.load_state_dict(checkpoint)
model.eval()
pred_sum = 0#model output
gt_sum = 0#label
tp_sum=0
fp_sum=0
fn_sum=0
acc=0
sum=0
result={}
with torch.no_grad():
    for it, (game_id,inputs,labels) in enumerate(test_loader):
        inputs=inputs.float()
        labels=labels
        output,_=model(inputs)
        TP,FP,TN,FN,pred_len, gt_len,pred=fmeasure(output.cpu(),labels.cpu())
        for idx,g in enumerate(game_id):
            if g not in result.keys():
                result[g]=pred[idx].tolist()
            else:
                result[g]+=pred[idx].tolist()
        print(TP,FP,TN,FN,pred_len, gt_len)
        tp_sum += TP
        fp_sum += FP
        fn_sum += FN
        pred_sum += pred_len
        gt_sum += gt_len
        acc=acc+TP+TN
        sum+=len(output)
    with open(weight_dir+'/train_result','a') as f:
        if tp_sum>0 and fp_sum>0 and fn_sum>0:
            precision = tp_sum/(tp_sum+fp_sum)
            recall = tp_sum / (tp_sum+fn_sum)
            f1 = (2*precision*recall / (precision + recall)) * 100
            accuracy=acc/sum
            print( tp_sum, fp_sum, fn_sum)
            print('[{}/{}], prec:{}, recall:{}, f1:{}, acc: {}'.format(it, len(test_loader), precision, recall, f1,accuracy))
            f.write('{}, prec:{}, recall:{}, f1:{}, acc : {}\n'.format(dataset, precision, recall, f1,accuracy))

data load fin
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
5 2 25 0 tensor(7) tensor(5)
8 0 23 1 tensor(8) tensor(9)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
9 4 19 0 tensor(13) tensor(9)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 16 16 tensor(0) tensor(16)
14 0 15 3 tensor(14) tensor(17)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
10 11 9 2 tensor(21) tensor(12)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 12 20 0 tensor(12) tensor(0)
0 3 29 0 tensor(3) tensor(0)
0 0 32 0 tensor(0) tensor(0)
7 0 25 0 tensor(7) tensor(7)
15 1 13 3 tensor(16) tensor(18)
6 3 21 2 tensor(9) tensor(8)
0 15 17 0 tensor(15) tensor(0)
11 0 19 2 ten

0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 10 22 0 tensor(10) tensor(0)
13 5 10 4 tensor(18) tensor(17)
8 5 14 5 tensor(13) tensor(13)
10 5 17 0 tensor(15) tensor(10)
0 0 32 0 tensor(0) tensor(0)
0 27 5 0 tensor(27) tensor(0)
0 15 17 0 tensor(15) tensor(0)
14 10 8 0 tensor(24) tensor(14)
4 0 21 7 tensor(4) tensor(11)
9 12 10 1 tensor(21) tensor(10)
0 0 30 2 tensor(0) tensor(2)
0 2 30 0 tensor(2) tensor(0)
0 13 19 0 tensor(13) tensor(0)
0 0 32 0 tensor(0) tensor(0)
15 8 9 0 tensor(23) tensor(15)
7 0 23 2 tensor(7) tensor(9)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 7 25 0 tensor(7) tensor(0)
0 10 22 0 tensor(10) tensor(0)
3 11 18 0 tensor(14) tensor(3)
27 0 0 5 tensor(27) tensor(32)
18 0 12 2 tensor(18) tensor(20)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
29 0 3 0 tensor(29) tensor(29)
8 7 13 4 tensor(15) tensor(12)
0 12 20 0 tensor(12) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 16

0 0 32 0 tensor(0) tensor(0)
0 16 16 0 tensor(16) tensor(0)
10 5 16 1 tensor(15) tensor(11)
0 15 17 0 tensor(15) tensor(0)
0 0 32 0 tensor(0) tensor(0)
9 0 23 0 tensor(9) tensor(9)
7 0 23 2 tensor(7) tensor(9)
0 6 26 0 tensor(6) tensor(0)
4 6 22 0 tensor(10) tensor(4)
0 24 8 0 tensor(24) tensor(0)
8 8 14 2 tensor(16) tensor(10)
11 4 16 1 tensor(15) tensor(12)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
3 7 22 0 tensor(10) tensor(3)
20 9 0 3 tensor(29) tensor(23)
8 11 12 1 tensor(19) tensor(9)
0 4 28 0 tensor(4) tensor(0)
0 5 27 0 tensor(5) tensor(0)
0 10 15 7 tensor(10) tensor(7)
0 0 26 6 tensor(0) tensor(6)
12 13 7 0 tensor(25) tensor(12)
17 0 12 3 tensor(17) tensor(20)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
9 8 15 0 tensor(17) tensor(9)
7 16 9 0 tensor(23) tensor(7)
0 1 29 2 tensor(1) tensor(2)
0 0 10 22 tensor(0) tensor(22)
26 3 3 0 tensor(29) tensor(26)
8 1 23 0 t

0 15 17 0 tensor(15) tensor(0)
0 3 29 0 tensor(3) tensor(0)
28 4 0 0 tensor(32) tensor(28)
3 14 13 2 tensor(17) tensor(5)
0 2 30 0 tensor(2) tensor(0)
23 4 5 0 tensor(27) tensor(23)
13 4 14 1 tensor(17) tensor(14)
6 4 22 0 tensor(10) tensor(6)
7 2 23 0 tensor(9) tensor(7)
29 1 2 0 tensor(30) tensor(29)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 8 24 0 tensor(8) tensor(0)
22 6 4 0 tensor(28) tensor(22)
7 7 18 0 tensor(14) tensor(7)
0 0 32 0 tensor(0) tensor(0)
6 6 20 0 tensor(12) tensor(6)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
11 0 20 1 tensor(11) tensor(12)
19 9 2 2 tensor(28) tensor(21)
8 8 16 0 tensor(16) tensor(8)
0 0 32 0 tensor(0) tensor(0)
12 0 16 4 tensor(12) tensor(16)
26 0 6 0 tensor(26) tensor(26)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 10 22 tensor(0) tensor(22)
22 0 3 7 ten

In [12]:
def fmeasure2(frames,label):
    average = [0,0,0,0,0]
    for key in frames.keys():
        TP = len(np.where((np.array(frames[key])==1)&(label[key]==1)==True)[0])
        FP = len(np.where((np.array(frames[key])==1)&(label[key]==0)==True)[0])
        TN = len(np.where((np.array(frames[key])==0)&(label[key]==0)==True)[0])
        FN = len(np.where((np.array(frames[key])==0)&(label[key]==1)==True)[0])
        precision = TP/(TP+FP)
        recall = TP/(TP+FN)
        accuracy = (TP+TN)/(TP+FN+FP+TN)
        if precision==0 and recall == 0:
            print('!')
        else:
            f1 = (2*precision*recall / (precision + recall))
            print(key)
            print('precision : {}, recall : {}, f1 : {}, accuracy : {}'.format(precision,recall,f1,accuracy))
            average[0]+= precision
            average[1]+= recall
            average[2]+= f1
            average[3]+= accuracy
            average[4]+=1
    print('==precision : {}, recall : {}, f1 : {}, accuracy : {}'.format(average[0]/average[4],average[1]/average[4],average[2]/average[4],average[3]/average[4]))
with open('./label/label.pickle',"rb") as f4:  
    real_result=pickle.load(f4)
fmeasure2(result,real_result)

102844212431058132
precision : 0.6177606177606177, recall : 0.8398950131233596, f1 : 0.7119021134593992, accuracy : 0.8948863636363636
102844341902586509
precision : 0.535796766743649, recall : 0.9169960474308301, f1 : 0.6763848396501457, accuracy : 0.8961646398503275
102844401152267937
precision : 0.6175496688741722, recall : 0.927860696517413, f1 : 0.7415506958250497, accuracy : 0.8864132809086938
102844212430927059
precision : 0.6617647058823529, recall : 0.8181818181818182, f1 : 0.7317073170731708, accuracy : 0.8958716802524322
102844412708953395
precision : 0.55, recall : 0.7971014492753623, f1 : 0.650887573964497, accuracy : 0.8852140077821011
102844212429944013
precision : 0.6386554621848739, recall : 0.8194070080862533, f1 : 0.717827626918536, accuracy : 0.8831867057673509
102844341912679064
precision : 0.5083333333333333, recall : 0.8097345132743363, f1 : 0.6245733788395904, accuracy : 0.8865979381443299
102844235753749959
precision : 0.563963963963964, recall : 0.796437659033

In [10]:
a=torch.transpose(b,1,2)

In [11]:
a

tensor([[[0.1380, 0.0111],
         [0.8294, 0.4059],
         [0.0521, 0.1911]]])

In [11]:
x=[]

In [9]:
with open('../data/chat_feature_pred_128_train.json',"rb") as f1:  
    chat_result=json.load(f1)

In [24]:
len(chat_result['102844235753356742'])

1654

In [21]:
        with open('../data/audio_energy_2_normaized.pickle',"rb") as f3:  
            audio_result=pickle.load(f3)

In [25]:
audio_result['102844235753356742'][1]

4.277879204882054e-07

In [8]:
        with open('../data/audio_H.pickle',"rb") as f2:  
            image_result=pickle.load(f2)

In [10]:
len(image_result['102844235753356742'])

16571